In [1]:
import requests
import itertools
import os
import yaml
import xarray as xr
import numpy as np

In [2]:
CURRENT_PATH = os.getcwd()
all_variables = yaml.safe_load(open(os.path.join(CURRENT_PATH, 'variables.yaml')))

In [3]:
def get_files_per_variable(chunk_i, variables):
    files_per_var = {}
    base_url = "https://prod-actris-md.nilu.no/metadata/content"
    n_variables = len(variables)
    for var_i, var in enumerate(variables):
        print(f'[{chunk_i}] {var} ({var_i}/{n_variables})')
        if var not in files_per_var:
            files_per_var[var] = {}
        variable_files = []
        page = 0
        while True:
            # Set up URL with pagination
            url = f"{base_url}/{var}/page/{page}"
            response = requests.get(url)
            
            # Check if the response is valid and contains data
            if response.status_code != 200:
                print(f"Error fetching page {page}. Status code: {response.status_code}")
                break
            
            data = response.json()
            
            # Check if there's content in the data
            if not data:
                break
            
            # Loop through each entry in the data and print DOI and OPeNDAP URL
            for item in data:
                doi = item.get("md_identification", {}).get("identifier", {}).get("pid")
                opendap_urls = [protocol_dict['dataset_url'] for protocol_dict in item.get('md_distribution_information', []) if protocol_dict.get('protocol') == 'OPeNDAP']
                
                # Print DOI and OPeNDAP URL if both are present
                if doi and opendap_urls:
                    variable_files.append(opendap_urls)
                    
            # Go to the next page
            page += 1
        
        files_per_var[var]['files'] = list(itertools.chain.from_iterable(variable_files))
    
    return files_per_var

In [4]:
files_per_var = get_files_per_variable(100, ['ozone amount concentration'])

[100] ozone amount concentration (0/1)


In [5]:
files_per_var

{'ozone amount concentration': {'files': []}}

In [6]:
#combined_data = {}
#chunk_size = 100
#chunks = [list(all_variables.keys())[i:i + chunk_size] for i in range(0, len(all_variables), chunk_size)]
#for chunk_i, chunk in enumerate(chunks):
#    files_per_var = get_files_per_variable(chunk_i, chunk)
#    combined_data.update(files_per_var)

In [7]:
#with open("ebas_doi_combined.yaml", "w") as file:
#    yaml.dump(combined_data, file, default_flow_style=False)

In [8]:
combined_data = yaml.safe_load(open(os.path.join(CURRENT_PATH, 'ebas_doi_combined.yaml')))

In [9]:
combined_data = {k: v for k, v in combined_data.items() if k.strip() and v}

In [10]:
variable_mapping = yaml.safe_load(open(os.path.join(CURRENT_PATH, 'variable_mapping.yaml')))
variable_mapping = {k: v for k, v in variable_mapping.items() if k.strip() and v}

In [11]:
actris_parameter = 'nitrogen dioxide mass concentration'
file = combined_data[actris_parameter]['files'][0]
ds = xr.open_dataset(file)
ds

<xarray.Dataset> Size: 21kB
Dimensions:                        (time: 365, tbnds: 2, metadata_time: 1,
                                    nitrogen_dioxide_qc_flags: 1,
                                    sulphur_dioxide_qc_flags: 1)
Coordinates:
  * time                           (time) datetime64[ns] 3kB 1990-01-01T19:00...
  * metadata_time                  (metadata_time) datetime64[ns] 8B 1990-07-...
Dimensions without coordinates: tbnds, nitrogen_dioxide_qc_flags,
                                sulphur_dioxide_qc_flags
Data variables:
    time_bnds                      (time, tbnds) datetime64[ns] 6kB ...
    metadata_time_bnds             (metadata_time, tbnds) datetime64[ns] 16B ...
    nitrogen_dioxide               (time) float64 3kB ...
    nitrogen_dioxide_qc            (nitrogen_dioxide_qc_flags, time) float64 3kB ...
    nitrogen_dioxide_ebasmetadata  (metadata_time) |S64 64B ...
    sulphur_dioxide                (time) float64 3kB ...
    sulphur_dioxide_qc             (sulphur_dioxide_qc_flags, time) float64 3kB ...
    sulphur_dioxide_ebasmetadata   (metadata_time) |S64 64B ...
Attributes: (12/85)
    Conventions:                   CF-1.8, ACDD-1.3
    featureType:                   timeSeries
    title:                         Inorganics in air and particle phase at La...
    keywords:                      nitrogen_dioxide, sulphur_dioxide, air, La...
    id:                            KJ5T-KCAB.nc
    naming_authority:              EBAS
    ...                            ...
    geospatial_lon_units:          degrees_east
    comment:                       {\n    "Data definition": "EBAS_1.1",\n   ...
    standard_name_vocabulary:      CF-1.7, ACDD-1.3
    history:                       None
    creator_url:                   ebas.nilu.no
    processing_level:              processing_level_test

In [12]:
ds = xr.open_dataset('https://thredds.nilu.no/thredds/dodsC/ebas_doi/KC/FH/JD/KCFH-JDKJ.nc')
ds

<xarray.Dataset> Size: 41kB
Dimensions:                        (time: 731, tbnds: 2, metadata_time: 2,
                                    nitrogen_dioxide_qc_flags: 1,
                                    sulphur_dioxide_qc_flags: 1)
Coordinates:
  * time                           (time) datetime64[ns] 6kB 1987-01-01T18:00...
  * metadata_time                  (metadata_time) datetime64[ns] 16B 1987-07...
Dimensions without coordinates: tbnds, nitrogen_dioxide_qc_flags,
                                sulphur_dioxide_qc_flags
Data variables:
    time_bnds                      (time, tbnds) datetime64[ns] 12kB ...
    metadata_time_bnds             (metadata_time, tbnds) datetime64[ns] 32B ...
    nitrogen_dioxide               (time) float64 6kB ...
    nitrogen_dioxide_qc            (nitrogen_dioxide_qc_flags, time) float64 6kB ...
    nitrogen_dioxide_ebasmetadata  (metadata_time) |S64 128B ...
    sulphur_dioxide                (time) float64 6kB ...
    sulphur_dioxide_qc             (sulphur_dioxide_qc_flags, time) float64 6kB ...
    sulphur_dioxide_ebasmetadata   (metadata_time) |S64 128B ...
Attributes: (12/85)
    Conventions:                   CF-1.8, ACDD-1.3
    featureType:                   timeSeries
    title:                         Inorganics in air and particle phase at Va...
    keywords:                      SE0011R, sulphur_dioxide, EMEP, air, Vavih...
    id:                            KCFH-JDKJ.nc
    naming_authority:              EBAS
    ...                            ...
    geospatial_lon_units:          degrees_east
    comment:                       {\n    "Data definition": "EBAS_1.1",\n   ...
    standard_name_vocabulary:      CF-1.7, ACDD-1.3
    history:                       None
    creator_url:                   ebas.nilu.no
    processing_level:              processing_level_test

In [13]:
# Define metadata keys, inspired in first dataset (TODO: Check all available metadata fields)
metadata_keys = {'ebas_station_code': 'station_reference',
                 'ebas_station_name': 'station_name',
                 'ebas_station_land_use': 'land_use',
                 'ebas_station_wmo_region': 'WMO_region',
                 'ebas_station_latitude': 'latitude', 
                 'ebas_station_longitude': 'longitude',
                 'ebas_station_altitude': 'altitude'}

parameters = {'nitrogen_dioxide': 'sconcno2'}

In [14]:
# open all files for certain variable, extract and join
resolution = 'hourly'
#for actris_parameter in ['ozone mass concentration']:
for actris_parameter in ['nitrogen dioxide mass concentration']:
    var = variable_mapping[actris_parameter]['var']
    print('ACTRIS vocabulary:', actris_parameter, '- Variable:', var)
    if len(combined_data[actris_parameter]['files']) != 0:
        # combine datasets that have the same variable
        combined_ds_list = []
        metadata = {}
        print('Total number of files:', len(combined_data[actris_parameter]['files']))
        for i, file in enumerate(combined_data[actris_parameter]['files'][0:50]):
            print(i, '-', file)
            # open file
            ds = xr.open_dataset(file)
                
            # assign station code as dimension
            ds = ds.expand_dims(dim={'station': [i]})
    
            # select data for that variable only
            if var not in ds:
                unformatted_units = variable_mapping[actris_parameter]['units']
                if unformatted_units == 'nmol/mol':
                    units = 'nmol_per_mol'
                elif unformatted_units == 'ug N/m3':
                    units = 'ug_N_per_m3'
                undocumented_var = f'{var}_{units}'
                ds_var = ds[undocumented_var]
            else:
                ds_var = ds[var]
            
            # save metadata
            for ebas_key in metadata_keys.keys():
                if ebas_key not in metadata.keys():
                    metadata[ebas_key] = []
                if ebas_key not in ds_var.attrs.keys():
                    metadata[ebas_key].append(np.nan)
                else:
                    metadata[ebas_key].append(ds_var.attrs[ebas_key])

            # remove all attributes except units
            ds_var.attrs = {key: value for key, value in ds_var.attrs.items() if key == 'units'}
                
            # append modified dataset to list
            combined_ds_list.append(ds_var)
    
        # combine and create new dataset
        combined_ds = xr.concat(combined_ds_list, 
                                dim='station', 
                                combine_attrs='drop_conflicts').to_dataset()

        # rename variable to BSC standards
        combined_ds = combined_ds.rename({var: parameters[var]})
                
        # add metadata
        for key, value in metadata.items():
            if metadata_keys[key] in ['latitude', 'longitude']:
                value = [float(val) for val in value]
            elif metadata_keys[key] == 'altitude':
                value = [float(val.replace('m', '').strip()) for val in value]
            combined_ds[metadata_keys[key]] = xr.Variable(data=value, dims=('station'))
        
        # save data per year and month
        path = f'/home/avilanov/data/providentia/obs/nonghost/actris/ebas_doi/{resolution}/{parameters[var]}'
        if not os.path.isdir(path):
            os.makedirs(path, exist_ok=True)
        for year, ds_year in combined_ds.groupby('time.year'):
            for month, ds_month in ds_year.groupby('time.month'):
                filename = f"{path}/{parameters[var]}_{year}{month:02d}.nc"
                combined_ds_yearmonth = combined_ds.sel(time=f"{year}-{month:02d}")
                combined_ds_yearmonth.to_netcdf(filename)
                print(f"Saved: {filename}")

ACTRIS vocabulary: nitrogen dioxide mass concentration - Variable: nitrogen_dioxide
Total number of files: 733
0 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/KJ/5T/KC/KJ5T-KCAB.nc
1 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/JK/E7/AA/JKE7-AABD.nc
2 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/BW/GE/SH/BWGE-SHXN.nc
3 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/JZ/S5/BN/JZS5-BNYJ.nc
4 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/KM/SZ/DN/KMSZ-DN7B.nc
5 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/R5/N5/TD/R5N5-TDVM.nc
6 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/QX/TN/E8/QXTN-E8CB.nc
7 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/2W/BJ/5E/2WBJ-5EAZ.nc
8 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/S5/MW/MR/S5MW-MR53.nc
9 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/2S/SQ/F2/2SSQ-F2FV.nc
10 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/9H/PS/FD/9HPS-FDMN.nc
11 - https://thredds.nilu.no/thredds/dodsC/ebas_doi/6R/SS/9S/6RSS-9S4S.nc
12 - https:

In [18]:
test_data = xr.open_dataset('/home/avilanov/data/providentia/obs/nonghost/actris/ebas_doi/hourly/sconcno2/sconcno2_202101.nc')
test_data

<xarray.Dataset> Size: 314kB
Dimensions:            (time: 744, station: 50)
Coordinates:
  * time               (time) datetime64[ns] 6kB 2021-01-01T00:30:00 ... 2021...
  * station            (station) int64 400B 0 1 2 3 4 5 6 ... 44 45 46 47 48 49
Data variables:
    sconcno2           (station, time) float64 298kB ...
    station_reference  (station) <U7 1kB ...
    station_name       (station) <U23 5kB ...
    land_use           (station) <U12 2kB ...
    WMO_region         (station) <U3 600B ...
    latitude           (station) float64 400B ...
    longitude          (station) float64 400B ...
    altitude           (station) float64 400B ...

In [16]:
nonghost_data = xr.open_dataset('/home/avilanov/data/providentia/obs/nonghost/eea/eionet/hourly/sconcno2/sconcno2_202406.nc')
nonghost_data

<xarray.Dataset> Size: 35MB
Dimensions:                       (station: 8643, time: 720)
Coordinates:
  * time                          (time) datetime64[ns] 6kB 2024-06-01 ... 20...
Dimensions without coordinates: station
Data variables: (12/20)
    station_code                  (station) |S75 648kB ...
    station_start_date            (station) |S75 648kB ...
    station_zone                  (station) |S75 648kB ...
    lat                           (station) float32 35kB ...
    street_type                   (station) |S75 648kB ...
    country_code                  (station) |S75 648kB ...
    ...                            ...
    lon                           (station) float32 35kB ...
    station_end_date              (station) |S75 648kB ...
    station_local_code            (station) |S75 648kB ...
    station_rural_back            (station) |S75 648kB ...
    station_ozone_classification  (station) |S75 648kB ...
    sconcno2                      (time, station) float32 25MB ...

In [17]:
ghost_data = xr.open_dataset('/home/avilanov/data/providentia/obs/ghost/EEA_AQ_eReporting/1.5/hourly/sconcno2/sconcno2_201805.nc')
ghost_data

<xarray.Dataset> Size: 3GB
Dimensions:                                                           (
                                                                       station: 3229,
                                                                       time: 744,
                                                                       N_flag_codes: 186,
                                                                       N_qa_codes: 79)
Coordinates:
  * time                                                              (time) datetime64[ns] 6kB ...
Dimensions without coordinates: station, N_flag_codes, N_qa_codes
Data variables: (12/183)
    ASTER_v3_altitude                                                 (station) float32 13kB ...
    EDGAR_v4.3.2_annual_average_BC_emissions                          (station) float32 13kB ...
    EDGAR_v4.3.2_annual_average_CO_emissions                          (station) float32 13kB ...
    EDGAR_v4.3.2_annual_average_NH3_emissions                         (station) float32 13kB ...
    EDGAR_v4.3.2_annual_average_NMVOC_emissions                       (station) float32 13kB ...
    EDGAR_v4.3.2_annual_average_NOx_emissions                         (station) float32 13kB ...
    ...                                                                ...
    station_timezone                                                  (station) <U18 232kB ...
    street_type                                                       (station) <U41 530kB ...
    street_width                                                      (station) float32 13kB ...
    terrain                                                           (station) <U8 103kB ...
    vertical_datum                                                    (station) <U20 258kB ...
    weekday_weekend_code                                              (station, time) uint8 2MB ...
Attributes:
    title:             Surface nitrogen dioxide data in the EEA_AQ_eReporting...
    institution:       Barcelona Supercomputing Center
    creator_name:      Dene R. Bowdalo
    creator_email:     dene.bowdalo@bsc.es
    source:            Observations
    version:           1.5
    application_area:  Monitoring atmospheric composition
    domain:            Atmosphere
    observed_layer:    Land surface
    data_licence:      BSD-3-Clause. Copyright 2023 Dene Bowdalo.